In [1]:
from requests import get
from bs4 import BeautifulSoup
from parsel import Selector
import pandas as pd
import numpy as np

# Send get request and get response from wikipedia

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = get(url)

# Use Selector and xpath to extract table data from response html

In [3]:
sel = Selector(response.text)
table = sel.xpath("//table[contains(@class,'wikitable')]/tbody/tr/td/text() | //table[contains(@class,'wikitable')]/tbody/tr/td/a/text()").extract()

# Transform data

In [90]:
sel = Selector(response.text)
table = sel.xpath("//table[contains(@class,'wikitable')]/tbody/tr/td/text() | //table[contains(@class,'wikitable')]/tbody/tr/td/a/text()").extract()
table = [x.replace('\n', '') for x in table if x != '\n']
table = np.reshape(table, (289,3))
table = [x for x in table if x[1] != 'Not assigned']
table = [[x[0],x[1],x[1]] if x[2] == 'Not assigned' else x for x in table]
    
df = pd.DataFrame(table, columns=['PostalCode', 'Borough', 'Neighborhood'])
dct = df.groupby('PostalCode').apply(lambda x: pd.Series({'Neighborhood':"{}".format(', '.join(x['Neighborhood']))}))
df = df.drop_duplicates(subset='PostalCode').drop('Neighborhood')
# df.shape

KeyError: "['Neighborhood'] not found in axis"

In [88]:
# print(df.update(dct))
# dct.reset_index(inplace=True)
# df.update(dct)

In [89]:
df.set_index('PostalCode').join(dct)

ValueError: columns overlap but no suffix specified: Index(['Neighborhood'], dtype='object')